In [ ]:
import pandas as pd
import os
import requests
import pgeocode
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET


In [2]:
# IGJ vragenlijsten mogelijk te specificiek, met specialisten tot een consensus komen over een algemene score. Custom algoritme?
# Mogelijk 

In [ ]:
# Download the data from the website
# The data is available at https://www.zorginzicht.nl/openbare-data
os.makedirs("data", exist_ok=True)
base_url = "https://www.zorginzicht.nl"
url = base_url + "/openbare-data"
page = requests.get(url)
soup = BeautifulSoup(page.content, "html.parser")
link_list = soup.find("ul", class_="link-list")
links = [a['href'].split("/")[2] for a in link_list.find_all("a", href=True)]
links = [link for link in links if "reval" in link]
links_c = [url + "/" + link for link in links]
# Remove links that contain "kwaliteitsstatuut"

for link in links_c:
    page = requests.get(link)
    soup = BeautifulSoup(page.content, "html.parser")
    # Find the div with class "page-module type-downloads"
    div = soup.find("div", class_="page-module type-downloads", id="verslagjaar-2023")
    link_file = base_url + div.find("a", class_ = "file-download icon-file-excel")['href']
    os.system(f'curl -o data/{"_".join(link.split("/")[-1].split("-")[2:])}.xlsx {link_file}')
    print(f'Downloaded {" ".join(link.split("/")[-1].split("-")[2:])}')

# Read the data from the downloaded files into a single DataFrame
# retrieve the second sheet from the file
dfs = {}
# Loop through all the files in the data directory and read the second sheet of each file into the df
for file in os.listdir("data"):
    if file.endswith(".xlsx"):
        # Read the sheet names from the file
        xls = pd.ExcelFile(os.path.join("data", file))
        sheet_names = xls.sheet_names
        print(sheet_names)
        # find the index of the sheet name that contains "2023"
        sheet_name = [sheet for sheet in sheet_names if "2023" in sheet]
        print(sheet_name)
        dfs[('_').join(file.split('.')[0].split('_')[2:])] = pd.read_excel(xls, sheet_name=sheet_name[0], header=0)
        
        # If the sheet name is not found, try reading the first sheet
        # Append the data to the df DataFrame
        #df = pd.concat([df, df_temp], ignore_index=True)
if len(dfs) == 1:
    dfs = dfs[list(dfs.keys())[0]]


dfs = dfs[dfs['IndicatorNaam'].str.contains('rapport')].reset_index(drop=True)
df = pd.DataFrame()
for Leeftijd in ['Jeugd','Volwassenen', 'Kinderen']:
    df_subset = dfs[dfs['IndicatorNaam'].str.contains('Volwassen')]
    df_subset = df_subset[df_subset['Indicator_nvt'] == 0]
    loc = []
    url = "https://zorgbeeld.nza.nl/openapi/WaitingTimeMSZ"
    for kvk,pc in zip(df_subset['KvkNummer'], df_subset['LocatiePostcode']):
        print(f'KVK: {kvk} - pc: {pc}')
        params = {
            "KVKNummer": kvk
        }

        response = requests.get(url, params=params)
        i = 0
        if response.status_code == 200:
            root = ET.fromstring(response.content)
            for entry in root.findall("TL_REST"):
                postal_code = entry.findtext("PostalCode")
                waiting_time = entry.findtext("WaitingTime")
                if postal_code == pc and waiting_time is not None:
                    loc.append(waiting_time)
                    print(f'{postal_code} - {waiting_time}')
                    i = 1
                    break
            if i == 0:
                print('No Match found')
                loc.append(None)
        else:
            print(f"Error: {response.status_code} for KVK {kvk}")
            loc.append(None)
    df_subset['loc'] = loc
    df = pd.concat([df, df_subset])

# Keep only the relevant columns
df.dropna(subset=['loc'], inplace=True)
df = pd.read_csv('WaitingTimes.csv', sep=',', encoding='utf-8')
df = df[["OrganisatieNaam", "LocatieNaam", "LocatiePlaats", "LocatiePostcode", "IndicatorWaarde", 'loc']]
df = df[df.duplicated() == False].reset_index(drop=True)
# take only the first 4 number of postal code
# rename loc column to WaitingTime
df.rename(columns={'loc': 'WaitingTime'}, inplace=True)
df['Postcode_kort'] = df['LocatiePostcode'].apply(lambda x: x[:4])
df['Postcode_kort'] = df['Postcode_kort'].astype(int)
df['WaitingTime'] = df['WaitingTime'].astype(int)
df.to_csv('WaitingTimes_2.csv', sep=',', encoding='utf-8', index=False)

TESTING 


In [ ]:
import pandas as pd
import pgeocode

# Load data
df = pd.read_csv('WaitingTimes_2.csv', sep=',', encoding='utf-8')
df['Postcode_kort'] = df['Postcode_kort'].astype(str).str[:4]  # Make sure it's string
geo = pgeocode.GeoDistance('NL')

def best_match():
    # Get postcode from query parameter
    postcode = 1183

    try:
        postcode_short = int(str(postcode)[:4])
    except ValueError:
        return 'error: Invalid postcode'

    # Compute distances to all locations
    distances = df['Postcode_kort'].apply(lambda x: geo.query_postal_code(str(postcode_short), str(x)))
    score1 = -distances / 25
    score2 = df['IndicatorWaarde']
    score3 = -df['WaitingTime'] / 5

    total_score = score1 + score2 + score3
    # Normalize the scores to a range of 0 to 1
    total_score = (total_score - total_score.min()) / (total_score.max() - total_score.min())

    top5_indices = total_score.nlargest(5).index
    matches = []

    for rank, i in enumerate(top5_indices, start=1):
        match = {
            '% match': float(total_score[i]),
            'Rank': int(rank),
            'OrganisatieNaam': str(df.iloc[i]['LocatieNaam']),
            'Postcode': str(df.iloc[i]['LocatiePostcode']),
            'Plaats': str(df.iloc[i]['LocatiePlaats']),
            'Kwaliteit': float(df.iloc[i]['IndicatorWaarde']),
            'WaitingTime': float(df.iloc[i]['WaitingTime']),
            'Distance': float(round(distances[i], 2)),
            'GoogleMapsLink': f"https://www.google.com/maps/dir/?api=1&origin={postcode}&destination={df.iloc[i]['LocatiePostcode']}"
        }
        matches.append(match)


    return matches

best_match()

[{'Rank': 1,
  'OrganisatieNaam': 'MRC Aardenburg',
  'Postcode': '3941PW',
  'Plaats': 'Doorn',
  'Kwaliteit': 8.9,
  'WaitingTime': 1.0,
  'Distance': 43.71,
  'GoogleMapsLink': 'https://www.google.com/maps/dir/?api=1&origin=1183&destination=3941PW'},
 {'Rank': 2,
  'OrganisatieNaam': 'Concernniveau: Stichting Medinello Revalidatie Zorg',
  'Postcode': '3821AL',
  'Plaats': 'Amersfoort',
  'Kwaliteit': 9.2,
  'WaitingTime': 7.0,
  'Distance': 39.76,
  'GoogleMapsLink': 'https://www.google.com/maps/dir/?api=1&origin=1183&destination=3821AL'},
 {'Rank': 3,
  'OrganisatieNaam': 'Heliomare Revalidatie (Wijk aan Zee)',
  'Postcode': '1949EC',
  'Plaats': 'Wijk aan Zee',
  'Kwaliteit': 8.3,
  'WaitingTime': 5.0,
  'Distance': 27.72,
  'GoogleMapsLink': 'https://www.google.com/maps/dir/?api=1&origin=1183&destination=1949EC'},
 {'Rank': 4,
  'OrganisatieNaam': 'St. Jansdal, concern',
  'Postcode': '3844DG',
  'Plaats': 'Harderwijk',
  'Kwaliteit': 8.5,
  'WaitingTime': 2.0,
  'Distance': 49.